<h1>1. Load ICESat-2 data from file</h1>

In [ ]:
import icepyx as ipx
%matplotlib inline
%matplotlib widget

Set output path

In [ ]:
# -----Determine where output files will be saved-----
output_path = '/ice_sat_2/'

# -----Manually enter Earthdata login info-----
earthdata_uid = input('Earthdata user ID: ')
email = input('Email: ')

Define search parameters

In [ ]:
# -----define search parameters-----
# ICESAT-2 data product name
short_name = 'ATL08'
# bounding box
spatial_extent = [-108.24005126953124, 38.884619201291905, -107.9, 39.11727568585598]
# date range
date_range = ['2020-09-01','2021-05-31']

# -----create the data object using our inputs-----
AOI_1 = ipx.Query(short_name, spatial_extent, date_range)

# -----log into Earthdata-----
AOI_1.earthdata_login(earthdata_uid, email)

# -----view query results-----
print('Query results:')
print(AOI_1.product)
print(AOI_1.dates)
print(AOI_1.start_time)
print(AOI_1.end_time)
print(AOI_1.cycles)
print(AOI_1.tracks)
print(AOI_1.product_version)
print('\nAvailable granules:')
print(AOI_1.avail_granules())
print(AOI_1.avail_granules(ids=True))

# -----build and view the parameters that will be submitted in query-----
print('CMR parameters:')
print(AOI_1.CMRparams)

# -----plot bounding box on map with approximate SnowEx location-----
AOI_1.visualize_spatial_extent()

Subset and Order granules

In [ ]:
# -----subset results-----
AOI_1.subsetparams()

# -----order granules-----
AOI_1.order_granules()

# -----view order IDs-----
print('\nOrder ID:')
AOI_1.granules.orderIDs

Download granules to output_path

In [1]:
AOI_1.download_granules(output_path)

NameError: name 'AOI_1' is not defined

<h1>2. Load DEMs</h1>

<h3>a. Copernicus</h2>

In [ ]:
CopDEM = rioxarray.open_rasterio('https://opentopography.s3.sdsc.edu/raster/COP30/COP30_hh.vrt', 
                              chunks=True, # ensure we use dask to only read metadata
                             )

# Crop by Bounding box of all the ATL08 points
minx, miny, maxx, maxy = gf.unary_union.envelope.bounds # got a depreciation warning with cascaded_union
CopDEM = CopDEM.rio.clip_box(minx, miny, maxx, maxy)


<h3>b. SRTM</h3>

In [3]:
import os
import ee
import pandas as pd
import geopandas as gpd
import geemap
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt
import h5py
from rasterio import plot, warp

In [ ]:
# -----GEE Authentication and Initialization
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

Extract SRTM elevations for AOI #1: Grand Mesa, CO

In [ ]:
# -----Create geojson AOI
# used geojson.io
AOI1 = ee.Geometry({"type": "Polygon","coordinates": 
                    [[[-108.24005126953124,38.884619201291905],
                      [-107.9,38.884619201291905],
                      [-107.9,39.11727568585598],
                      [-108.24005126953124,39.11727568585598],
                      [-108.24005126953124,38.884619201291905]]
                    ]})

# -----Query for SRTM, clip to AOI
SRTM1 = ee.Image("USGS/SRTMGL1_003").clip(AOI1)

# -----Plot elevations
# from: https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api
SRTM1_url = SRTM1.getThumbUrl({
    'min': 0, 'max': 6000,
    'palette': ['white', 'green']})
Image(url=SRTM1_url)

In [ ]:
# -----Load snow-off ICESat-2 track
data_path = '/home/jovyan/data/'
is2_file = 'processed_ATL08_20200920013522_13240806_005_01.h5' # Sept = snow-off 

with h5py.File(data_path+is2_file, 'r') as f:
    is2_gt2l = pd.DataFrame(data={'lat': f['gt2l/land_segments/latitude'][:],
                                 'lon': f['gt2l/land_segments/longitude'][:],
                                  'h_te_best_fit': f['gt2l/land_segments/terrain/h_te_best_fit'][:]})

# select one of the beams and convert it to a geodataframe
is2_gdf = gpd.GeoDataFrame(is2_gt2l, geometry=gpd.points_from_xy(is2_gt2l.lon, is2_gt2l.lat), crs='epsg:7661')
is2_gdf.head()

In [ ]:
# -----function to convert pandas GeoDataFrame to ee.FeatureCollection
from functools import reduce
def feature2ee(gdf):
    g = [i for i in gdf.geometry]
    features=[]
    #for Point geo data type
    if (gdf.geom_type[0] == 'Point'):
        for i in range(len(g)):
            g = [i for i in gdf.geometry]
            x,y = g[i].coords.xy
            cords = np.dstack((x,y)).tolist()
            double_list = reduce(lambda x,y: x+y, cords)
            single_list = reduce(lambda x,y: x+y, double_list)
            g=ee.Geometry.Point(single_list)
            feature = ee.Feature(g)
            features.append(feature)
        ee_object = ee.FeatureCollection(features)
        return ee_object

In [ ]:
# -----convert ICESat-2 gdf to ee.FeatureCollection
is2_ee = feature2ee(is2_gdf)
is2_ee.getInfo();

# -----sample SRTM elevation at ICESat-2 coordinates
SRTM2_sample = ee.FeatureCollection(SRTM2.sample(is2_ee))
SRTM2_sample_gdf = geemap.ee_to_geopandas(SRTM2_sample, selectors=['elevation'])
SRTM2_sample_gdf['lon'] = is2_gdf['lon']
SRTM2_sample_gdf['lat'] = is2_gdf['lat']

# -----calculate elevation difference
snow_off_diff = is2_gdf['h_te_best_fit'] - SRTM2_sample_gdf['elevation']

In [ ]:
# -----plot
# SRTM & ICESat-2 elevations
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))
ax1.plot(SRTM2_sample_gdf['lat'], 
         SRTM2_sample_gdf['elevation'], color='blue', label='SRTM')
ax1.plot(SRTM2_sample_gdf['lat'],
         is2_gdf['h_te_best_fit'], color='orange', label='ICESat-2')
ax1.set_ylabel('elevation [m]')
ax1.grid()
ax1.legend()
# elevation difference
ax2.plot(SRTM2_sample_gdf['lat'], snow_off_diff, color='black', label='difference ("snow depth")')
ax2.set_xlabel('Latitude')
ax2.set_ylabel('elevation [m]')
ax2.legend()
ax2.grid()
plt.show()

# -----save figure
fig.savefig('../figures/snow_free_profile_SRTM_processed_ATL08_20200920013522_13240806_005_01.png',dpi=200)
print('figure saved to file')

<h1>3. Coregister</h1>

<h1>4. Estimate snow depth for each ICESat-2 track</h1>

<h1>5. Load SNOTEL snow depths</h1>

In [ ]:
import os
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import contextily as ctx
import ulmo
import hvplot.pandas
from geoviews import tile_sources as gvts

In [ ]:
# -----latest CUAHSI API endpoint
wsdlurl = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'

# -----fetch available site metadata from server
sites = ulmo.cuahsi.wof.get_sites(wsdlurl)

# -----store dictionary as Pandas DataFrame
sites_df = pd.DataFrame.from_dict(sites, orient='index').dropna()
sites_df.head()

# ---clean up and convert to geodataframe - as in geospatial tutorial 
# get the lat/lon values as columns rather than a dict
sites_df['lat'] = [float(loc['latitude']) for loc in sites_df['location']]
sites_df['lon'] = [float(loc['longitude']) for loc in sites_df['location']]
# convert to geodataframe, using the lat/lon columns
sites_df = gpd.GeoDataFrame(sites_df, geometry=gpd.points_from_xy(sites_df.lon, sites_df.lat), crs='epsg:7661')
# drop 'location' column (no longer needed)
sites_df = sites_df.drop(columns={'location','lat','lon'})
# set elevation column datatype to float
sites_df = sites_df.astype({"elevation_m":float})
# sort DataFrame by name
sites_df = sites_df.sort_values(by='name')

# -----view all sites (very big)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
#     print(sites_df)
    
# -----filter to study sites
# Grand Mesa (GM) - two SNOTEL sites
GM1 = sites_df.loc['SNOTEL:622_CO_SNTL']
GM2 = sites_df.loc['SNOTEL:682_CO_SNTL']
# Mores Creek Summit (MCS)
MCS = sites_df.loc['SNOTEL:637_ID_SNTL']

sites_df.head()

In [ ]:
# -----function to fetch SNOTEL snow depth
def snotel_fetch(sitecode, start_date, end_date):
    variablecode='SNOTEL:SNWD_D'
    #print(sitecode, variablecode, start_date, end_date)
    values_df = None
    try:
        #Request data from the server
        site_values = ulmo.cuahsi.wof.get_values(wsdlurl, sitecode, variablecode, start=start_date, end=end_date)
        #Convert to a Pandas DataFrame   
        values_df = pd.DataFrame.from_dict(site_values['values'])
        #Parse the datetime values to Pandas Timestamp objects
        values_df['datetime'] = pd.to_datetime(values_df['datetime'], utc=True)
        #Set the DataFrame index to the Timestamps
        values_df = values_df.set_index('datetime')
        #Convert values to float and replace -9999 nodata values with NaN
        values_df['value'] = pd.to_numeric(values_df['value']).replace(-9999, np.nan)
        #Remove any records flagged with lower quality
        values_df = values_df[values_df['quality_control_level_code'] == '1']
    except:
        print("Unable to fetch %s" % variablecode)
    
    return values_df

# -----set start and end date
start_date = '2020-09-01'
end_date = '2021-05-31'

# -----fetch data
GM1_sd = snotel_fetch(sitecode=GM1['code'], start_date=start_date, end_date=end_date)
GM2_sd = snotel_fetch(sitecode=GM2['code'], start_date=start_date, end_date=end_date)
MCS_sd = snotel_fetch(sitecode=MCS['code'], start_date=start_date, end_date=end_date)

In [ ]:
# -----quick plot
fig = plt.figure(figsize=(10, 12))
plt.rcParams.update({'font.size': 12, 'font.serif': 'Arial'})
# ax1 - snow depth
ax1 = fig.add_subplot(2, 1, 1)
ax1.plot(GM1_sd['value'], label=GM1['name']+' (Grand Mesa)', color='blue')
ax1.plot(GM2_sd['value'], label=GM2['name']+' (Grand Mesa)', color='green')
ax1.plot(MCS_sd['value'], label=MCS['name'], color='orange')
ax1.grid()
ax1.legend()
ax1.set_ylabel('snow depth [cm]')
# ax2 - coordinates
ax2 = fig.add_subplot(2, 1, 2)
ax2.scatter(GM1['geometry'].x, GM1['geometry'].y, label=GM1['name']+' (Grand Mesa)', color='blue', s=20)
ax2.scatter(GM2['geometry'].x, GM2['geometry'].y, label=GM2['name']+' (Grand Mesa)', color='green', s=20)
ax2.scatter(MCS['geometry'].x, MCS['geometry'].y, label=MCS['name'], color='orange', s=20)
# shaded relief background, for a coloured topo map: don't specify any source in the line below
ctx.add_basemap(ax=ax2, crs=sites_df.crs, source=ctx.providers.Esri.WorldShadedRelief) 
ax2.grid()
ax2.set_xlabel('lon')
ax2.set_ylabel('lat')
ax2.legend(loc='right')

# -----export figure
fig.savefig('../figures/SNOTEL_snowdepth.png', dpi=300)
print('figure saved to file')

<h1>6. Compare results</h1>